# Strategy 1: Advanced Residual + Selective Attention

**Based on Professor's Top Submissions (Avg Score: 77.8)**

## Architecture Features:
- Residual connections with skip paths
- Selective multi-head attention (1 attention block)
- LayerNorm + BatchNorm combinations
- Log-space prediction (log2)
- Progressive dropout (0.3 → 0.2 → 0.1)
- AdamW optimizer with weight decay

## Data Strategy:
- **CRITICAL: Split FIRST, then augment ONLY training data**
- Heavy augmentation on training set: 3x multiplier
- Techniques: Gaussian noise, perturbations, SMOTE-like interpolation
- Validation set: UNTOUCHED (no augmentation)

In [1]:
import numpy as np
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import (
    Input, Dense, Embedding, Flatten, Concatenate,
    Dropout, BatchNormalization, LayerNormalization, Add,
    Reshape, MultiHeadAttention, Lambda
)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import (
    EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
)
from tensorflow.keras.optimizers import AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("="*70)
print("STRATEGY 1: ADVANCED RESIDUAL + SELECTIVE ATTENTION")
print("="*70)
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

STRATEGY 1: ADVANCED RESIDUAL + SELECTIVE ATTENTION
TensorFlow version: 2.19.0
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Step 1: Load Data

In [2]:
print("\nSTEP 1: Loading Data")
print("-"*70)

with open('combined_final_n_k_m_P.pkl', 'rb') as f:
    inputs_raw = pickle.load(f)

with open('combined_final_mHeights.pkl', 'rb') as f:
    outputs_raw = pickle.load(f)

print(f"Raw samples: {len(inputs_raw)}")
print(f"Sample: n={inputs_raw[0][0]}, k={inputs_raw[0][1]}, m={inputs_raw[0][2]}, P={inputs_raw[0][3].shape}")
print(f"Target range: [{np.min(outputs_raw):.2f}, {np.max(outputs_raw):.2f}]")


STEP 1: Loading Data
----------------------------------------------------------------------
Raw samples: 108000
Sample: n=9, k=5, m=3, P=(5, 4)
Target range: [2.00, 33517570.00]


## Step 2: Split Data FIRST (NO AUGMENTATION YET)

In [3]:
print("\nSTEP 2: Split Data FIRST (before augmentation)")
print("-"*70)

# Create stratification labels
stratify_labels = [sample[1] * 10 + sample[2] for sample in inputs_raw]

# Split FIRST
inputs_train, inputs_val, outputs_train, outputs_val = train_test_split(
    inputs_raw, outputs_raw,
    test_size=0.15,
    random_state=42,
    stratify=stratify_labels
)

print(f"Training samples (before augmentation): {len(inputs_train)}")
print(f"Validation samples (NO augmentation): {len(inputs_val)}")
print("\n⚠️  CRITICAL: Validation data will NOT be augmented (prevents data leakage)")


STEP 2: Split Data FIRST (before augmentation)
----------------------------------------------------------------------
Training samples (before augmentation): 91800
Validation samples (NO augmentation): 16200

⚠️  CRITICAL: Validation data will NOT be augmented (prevents data leakage)


## Step 3: Augment ONLY Training Data (3x)

In [4]:
print("\nSTEP 3: Augment ONLY Training Data (3x multiplier)")
print("-"*70)

def augment_sample_gaussian(n, k, m, P, target, noise_level=0.03):
    P_aug = P.copy().astype(np.float32)
    noise = np.random.normal(0, noise_level, P_aug.shape)
    P_aug = P_aug + noise * np.std(P_aug)
    return [n, k, m, P_aug], target

def augment_sample_perturbation(n, k, m, P, target, strength=0.02):
    P_aug = P.copy().astype(np.float32)
    perturbation = np.random.uniform(-strength, strength, P_aug.shape)
    P_aug = P_aug * (1 + perturbation)
    return [n, k, m, P_aug], target

def augment_sample_interpolation(sample1, sample2, target1, target2):
    alpha = np.random.uniform(0.3, 0.7)
    n, k, m = sample1[0], sample1[1], sample1[2]
    P1, P2 = sample1[3], sample2[3]
    if sample1[1] == sample2[1] and sample1[2] == sample2[2]:
        P_new = alpha * P1.astype(np.float32) + (1 - alpha) * P2.astype(np.float32)
        target_new = alpha * target1 + (1 - alpha) * target2
        return [n, k, m, P_new], target_new
    return None, None

# Group TRAINING samples by (k,m)
train_groups = defaultdict(list)
for i, sample in enumerate(inputs_train):
    k, m = sample[1], sample[2]
    train_groups[(k, m)].append(i)

inputs_train_aug = []
outputs_train_aug = []

# Augment ONLY training data
for i, (sample, target) in enumerate(zip(inputs_train, outputs_train)):
    n, k, m, P = sample

    # Keep original
    inputs_train_aug.append(sample)
    outputs_train_aug.append(target)

    # Aug 1: Gaussian noise
    aug1, tgt1 = augment_sample_gaussian(n, k, m, P, target)
    inputs_train_aug.append(aug1)
    outputs_train_aug.append(tgt1)

    # Aug 2: Perturbation or interpolation
    if np.random.rand() < 0.5:
        aug2, tgt2 = augment_sample_perturbation(n, k, m, P, target)
        inputs_train_aug.append(aug2)
        outputs_train_aug.append(tgt2)
    else:
        group_indices = train_groups[(k, m)]
        if len(group_indices) > 1:
            j = np.random.choice([idx for idx in group_indices if idx != i])
            aug2, tgt2 = augment_sample_interpolation(
                sample, inputs_train[j], target, outputs_train[j]
            )
            if aug2 is not None:
                inputs_train_aug.append(aug2)
                outputs_train_aug.append(tgt2)
            else:
                aug2, tgt2 = augment_sample_perturbation(n, k, m, P, target)
                inputs_train_aug.append(aug2)
                outputs_train_aug.append(tgt2)
        else:
            aug2, tgt2 = augment_sample_perturbation(n, k, m, P, target)
            inputs_train_aug.append(aug2)
            outputs_train_aug.append(tgt2)

print(f"Training samples after augmentation: {len(inputs_train_aug)}")
print(f"Augmentation ratio: {len(inputs_train_aug) / len(inputs_train):.2f}x")
print(f"Validation samples (unchanged): {len(inputs_val)}")
print("\n✅ NO DATA LEAKAGE: Validation set is completely independent")


STEP 3: Augment ONLY Training Data (3x multiplier)
----------------------------------------------------------------------
Training samples after augmentation: 275400
Augmentation ratio: 3.00x
Validation samples (unchanged): 16200

✅ NO DATA LEAKAGE: Validation set is completely independent


## Step 4: Prepare Data for Training

In [5]:
print("\nSTEP 4: Prepare Data for Training")
print("-"*70)

def prepare_data(inputs, outputs):
    n_vals = []
    k_vals = []
    m_vals = []
    P_flat = []

    for sample in inputs:
        n_vals.append(sample[0])
        k_vals.append(sample[1])
        m_vals.append(sample[2])
        P_flat.append(sample[3].flatten())

    n_vals = np.array(n_vals, dtype=np.float32).reshape(-1, 1)
    k_vals = np.array(k_vals, dtype=np.int32).reshape(-1, 1)
    m_vals = np.array(m_vals, dtype=np.int32).reshape(-1, 1)
    outputs_arr = np.array(outputs, dtype=np.float32)

    # Pad P matrices
    max_p_size = max(len(p) for p in P_flat)
    P_padded = []
    for p in P_flat:
        if len(p) < max_p_size:
            padded = np.zeros(max_p_size, dtype=np.float32)
            padded[:len(p)] = p
            P_padded.append(padded)
        else:
            P_padded.append(p)

    P_arr = np.array(P_padded, dtype=np.float32)
    outputs_arr = np.maximum(outputs_arr, 1.0)

    return n_vals, k_vals, m_vals, P_arr, outputs_arr

n_train, k_train, m_train, P_train, y_train = prepare_data(inputs_train_aug, outputs_train_aug)
n_val, k_val, m_val, P_val, y_val = prepare_data(inputs_val, outputs_val)

# Normalize P matrices (fit on training, transform both)
scaler = StandardScaler()
P_train = scaler.fit_transform(P_train)
P_val = scaler.transform(P_val)  # Use training scaler

print(f"Training: n={n_train.shape}, k={k_train.shape}, m={m_train.shape}, P={P_train.shape}, y={y_train.shape}")
print(f"Validation: n={n_val.shape}, k={k_val.shape}, m={m_val.shape}, P={P_val.shape}, y={y_val.shape}")
print(f"P train: mean={P_train.mean():.4f}, std={P_train.std():.4f}")
print(f"P val: mean={P_val.mean():.4f}, std={P_val.std():.4f}")


STEP 4: Prepare Data for Training
----------------------------------------------------------------------
Training: n=(275400, 1), k=(275400, 1), m=(275400, 1), P=(275400, 20), y=(275400,)
Validation: n=(16200, 1), k=(16200, 1), m=(16200, 1), P=(16200, 20), y=(16200,)
P train: mean=0.0000, std=1.0000
P val: mean=0.0015, std=1.0427


## Step 5: Build Model

In [6]:
print("\nSTEP 5: Building Advanced Residual + Attention Model")
print("-"*70)

def build_model(p_shape, k_vocab_size=7, m_vocab_size=6):
    n_input = Input(shape=(1,), name='n')
    k_input = Input(shape=(1,), name='k', dtype=tf.int32)
    m_input = Input(shape=(1,), name='m', dtype=tf.int32)
    P_input = Input(shape=(p_shape,), name='P_flat')

    k_embed = Flatten()(Embedding(k_vocab_size, 32)(k_input))
    m_embed = Flatten()(Embedding(m_vocab_size, 32)(m_input))

    x = Dense(256, activation='gelu')(P_input)
    x = LayerNormalization()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    x = Dense(512, activation='gelu')(x)
    x = LayerNormalization()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    x_attn = Reshape((1, 512))(x)
    x_attn = MultiHeadAttention(num_heads=4, key_dim=64, dropout=0.1)(x_attn, x_attn)
    x_attn = Flatten()(x_attn)

    combined = Concatenate()([n_input, k_embed, m_embed, x_attn])

    x = Dense(1024, activation='gelu')(combined)
    x = LayerNormalization()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    residual = x
    x = Dense(1024, activation='gelu')(x)
    x = LayerNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(1024, activation='gelu')(x)
    x = LayerNormalization()(x)
    x = Add()([x, residual])

    residual = x
    x = Dense(1024, activation='gelu')(x)
    x = LayerNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(1024, activation='gelu')(x)
    x = LayerNormalization()(x)
    x = Add()([x, residual])

    x = Dense(512, activation='gelu')(x)
    x = LayerNormalization()(x)
    x = Dropout(0.2)(x)

    x = Dense(256, activation='gelu')(x)
    x = LayerNormalization()(x)
    x = Dropout(0.1)(x)

    x = Dense(128, activation='gelu')(x)
    x = LayerNormalization()(x)
    x = Dropout(0.1)(x)

    log2_pred = Dense(1, activation='linear')(x)
    log2_positive = Lambda(lambda z: tf.nn.softplus(z))(log2_pred)
    m_height = Lambda(lambda z: tf.pow(2.0, z))(log2_positive)

    return Model(inputs=[n_input, k_input, m_input, P_input], outputs=m_height,
                 name='strategy1_advanced_residual_attention')

model = build_model(P_train.shape[1], k_vocab_size=k_train.max()+1, m_vocab_size=m_train.max()+1)
print(f"Parameters: {model.count_params():,}")
model.summary()


STEP 5: Building Advanced Residual + Attention Model
----------------------------------------------------------------------
Parameters: 6,163,105


Model: "strategy1_advanced_residual_attention"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ P_flat (InputLayer) │ (None, 20)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │      5,376 │ P_flat[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 256)       │        512 │ dense[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256)       │      1,024 │ layer_normalizat… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 512)       │    131,584 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 512)       │      1,024 │ dense_1[0][0]     │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 512)       │      2,048 │ layer_normalizat… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 512)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ k (InputLayer)      │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ m (InputLayer)      │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 1, 512)    │          0 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 1, 32)     │        224 │ k[0][0]           │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 1, 32)     │        192 │ m[0][0]           │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 1, 512)    │    525,568 │ reshape[0][0],    │
│ (MultiHeadAttentio… │                   │            │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ n (InputLayer)      │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 32)        │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 32)        │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 512)       │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 577)       │          0 │ n[0][0],          │
│ (Concatenate)       │                   │            │ flatten[0][0],    │
│                     │                   │            │ flatten_1[0][0],  │
│                     │                   │            │ flatten_2[0][0] 

 Total params: 6,163,105 (23.51 MB)

 Trainable params: 6,159,521 (23.50 MB)

 Non-trainable params: 3,584 (14.00 KB)

## Step 6: Train

In [7]:
print("\nSTEP 6: Compile and Train")
print("-"*70)

def log2_mse_loss(y_true, y_pred):
    epsilon = 1e-7
    y_true = tf.maximum(y_true, epsilon)
    y_pred = tf.maximum(y_pred, epsilon)
    log2_true = tf.math.log(y_true) / tf.math.log(2.0)
    log2_pred = tf.math.log(y_pred) / tf.math.log(2.0)
    return tf.reduce_mean(tf.square(log2_true - log2_pred))

optimizer = AdamW(learning_rate=1e-3, weight_decay=1e-4, clipnorm=1.0)
model.compile(optimizer=optimizer, loss=log2_mse_loss, metrics=[log2_mse_loss])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=15, min_lr=1e-6, verbose=1),
    ModelCheckpoint('strategy1_best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
]

history = model.fit(
    [n_train, k_train, m_train, P_train], y_train,
    validation_data=([n_val, k_val, m_val, P_val], y_val),
    epochs=200, batch_size=256, callbacks=callbacks, verbose=1
)

print("\nTraining completed!")


STEP 6: Compile and Train
----------------------------------------------------------------------
Epoch 1/200
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - log2_mse_loss: 17.8421 - loss: 4.5582
Epoch 1: val_loss improved from inf to 1.32258, saving model to strategy1_best_model.h5


1076/1076 ━━━━━━━━━━━━━━━━━━━━ 55s 26ms/step - log2_mse_loss: 17.8418 - loss: 4.5559 - val_log2_mse_loss: 17.8110 - val_loss: 1.3226 - learning_rate: 0.0010
Epoch 2/200
1071/1076 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - log2_mse_loss: 17.6144 - loss: 1.4326
Epoch 2: val_loss improved from 1.32258 to 1.30018, saving model to strategy1_best_model.h5


1076/1076 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - log2_mse_loss: 17.6143 - loss: 1.4326 - val_log2_mse_loss: 17.6528 - val_loss: 1.3002 - learning_rate: 0.0010
Epoch 3/200
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - log2_mse_loss: 17.6031 - loss: 1.3923
Epoch 3: val_loss improved from 1.30018 to 1.28411, saving model to strategy1_best_model.h5


1076/1076 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - log2_mse_loss: 17.6031 - loss: 1.3923 - val_log2_mse_loss: 17.9240 - val_loss: 1.2841 - learning_rate: 0.0010
Epoch 4/200
1070/1076 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - log2_mse_loss: 17.6128 - loss: 1.3808
Epoch 4: val_loss improved from 1.28411 to 1.27874, saving model to strategy1_best_model.h5


1076/1076 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - log2_mse_loss: 17.6127 - loss: 1.3808 - val_log2_mse_loss: 18.0834 - val_loss: 1.2787 - learning_rate: 0.0010
Epoch 5/200
1074/1076 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - log2_mse_loss: 17.6166 - loss: 1.3678
Epoch 5: val_loss did not improve from 1.27874
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - log2_mse_loss: 17.6166 - loss: 1.3678 - val_log2_mse_loss: 18.0159 - val_loss: 1.2933 - learning_rate: 0.0010
Epoch 6/200
1071/1076 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - log2_mse_loss: 17.6190 - loss: 1.3592
Epoch 6: val_loss improved from 1.27874 to 1.26303, saving model to strategy1_best_model.h5


1076/1076 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - log2_mse_loss: 17.6189 - loss: 1.3592 - val_log2_mse_loss: 17.9267 - val_loss: 1.2630 - learning_rate: 0.0010
Epoch 7/200
1075/1076 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - log2_mse_loss: 17.6104 - loss: 1.3472
Epoch 7: val_loss did not improve from 1.26303
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - log2_mse_loss: 17.6104 - loss: 1.3472 - val_log2_mse_loss: 18.1672 - val_loss: 1.2828 - learning_rate: 0.0010
Epoch 8/200
1073/1076 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - log2_mse_loss: 17.6183 - loss: 1.3479
Epoch 8: val_loss did not improve from 1.26303
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - log2_mse_loss: 17.6182 - loss: 1.3479 - val_log2_mse_loss: 18.0283 - val_loss: 1.2634 - learning_rate: 0.0010
Epoch 9/200
1074/1076 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - log2_mse_loss: 17.6232 - loss: 1.3300
Epoch 9: val_loss did not improve from 1.26303
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - log2_mse_loss: 17.6232 - loss: 1.3300 - val_log2_mse_loss: 17.91

1076/1076 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - log2_mse_loss: 17.6332 - loss: 1.3106 - val_log2_mse_loss: 17.8516 - val_loss: 1.2616 - learning_rate: 0.0010
Epoch 13/200
1073/1076 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - log2_mse_loss: 17.6307 - loss: 1.3010
Epoch 13: val_loss did not improve from 1.26157
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - log2_mse_loss: 17.6306 - loss: 1.3010 - val_log2_mse_loss: 17.8795 - val_loss: 1.2637 - learning_rate: 0.0010
Epoch 14/200
1074/1076 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - log2_mse_loss: 17.6321 - loss: 1.2932
Epoch 14: val_loss improved from 1.26157 to 1.25113, saving model to strategy1_best_model.h5


1076/1076 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - log2_mse_loss: 17.6321 - loss: 1.2932 - val_log2_mse_loss: 17.9213 - val_loss: 1.2511 - learning_rate: 0.0010
Epoch 15/200
1075/1076 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - log2_mse_loss: 17.6335 - loss: 1.2883
Epoch 15: val_loss did not improve from 1.25113
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - log2_mse_loss: 17.6335 - loss: 1.2883 - val_log2_mse_loss: 18.0223 - val_loss: 1.2582 - learning_rate: 0.0010
Epoch 16/200
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - log2_mse_loss: 17.6381 - loss: 1.2811
Epoch 16: val_loss improved from 1.25113 to 1.24925, saving model to strategy1_best_model.h5


1076/1076 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - log2_mse_loss: 17.6381 - loss: 1.2811 - val_log2_mse_loss: 17.9115 - val_loss: 1.2492 - learning_rate: 0.0010
Epoch 17/200
1071/1076 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - log2_mse_loss: 17.6365 - loss: 1.2714
Epoch 17: val_loss improved from 1.24925 to 1.24921, saving model to strategy1_best_model.h5


1076/1076 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - log2_mse_loss: 17.6365 - loss: 1.2714 - val_log2_mse_loss: 18.0804 - val_loss: 1.2492 - learning_rate: 0.0010
Epoch 18/200
1070/1076 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - log2_mse_loss: 17.6512 - loss: 1.2666
Epoch 18: val_loss did not improve from 1.24921
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - log2_mse_loss: 17.6512 - loss: 1.2665 - val_log2_mse_loss: 17.7229 - val_loss: 1.2593 - learning_rate: 0.0010
Epoch 19/200
1071/1076 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - log2_mse_loss: 17.6503 - loss: 1.2519
Epoch 19: val_loss did not improve from 1.24921
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - log2_mse_loss: 17.6503 - loss: 1.2519 - val_log2_mse_loss: 17.9690 - val_loss: 1.2562 - learning_rate: 0.0010
Epoch 20/200
1074/1076 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - log2_mse_loss: 17.6543 - loss: 1.2414
Epoch 20: val_loss did not improve from 1.24921
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - log2_mse_loss: 17.6544 - loss: 1.2414 - val_log2_mse_loss:

## Step 7: Evaluate

In [8]:
print("\nSTEP 7: Evaluation")
print("-"*70)

model.load_weights('strategy1_best_model.h5')
y_pred_val = model.predict([n_val, k_val, m_val, P_val], verbose=0).flatten()

def compute_log2_mse(y_true, y_pred):
    epsilon = 1e-7
    y_true = np.maximum(y_true, epsilon)
    y_pred = np.maximum(y_pred, epsilon)
    return np.mean((np.log2(y_true) - np.log2(y_pred)) ** 2)

val_log2_mse = compute_log2_mse(y_val, y_pred_val)

print(f"\nValidation log2-MSE: {val_log2_mse:.6f}")
print(f"Prediction range: [{y_pred_val.min():.2f}, {y_pred_val.max():.2f}]")

group_metrics = defaultdict(lambda: {'true': [], 'pred': []})
for i in range(len(y_val)):
    k, m = k_val[i, 0], m_val[i, 0]
    group_metrics[(k, m)]['true'].append(y_val[i])
    group_metrics[(k, m)]['pred'].append(y_pred_val[i])

print("\nPer-Group Performance:")
print(f"{'Group':<12} {'n_val':<8} {'log2-MSE':<12}")
print("-"*40)
for (k, m), data in sorted(group_metrics.items()):
    group_mse = compute_log2_mse(np.array(data['true']), np.array(data['pred']))
    print(f"k={k}, m={m}    {len(data['true']):6d}   {group_mse:.6f}")

print("\n" + "="*70)
print("STRATEGY 1 COMPLETE")
print("="*70)


STEP 7: Evaluation
----------------------------------------------------------------------

Validation log2-MSE: 1.249205
Prediction range: [5.45, 29614.96]

Per-Group Performance:
Group        n_val    log2-MSE    
----------------------------------------
k=4, m=2      1800   0.292279
k=4, m=3      1800   0.349115
k=4, m=4      1800   0.839172
k=4, m=5      1800   2.630090
k=5, m=2      1800   0.308730
k=5, m=3      1800   0.892862
k=5, m=4      1800   2.622234
k=6, m=2      1800   0.702601
k=6, m=3      1800   2.605762

STRATEGY 1 COMPLETE
